In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

,"""'departwb'""","""'departdaytype'""","""'fetchdays'""","""'fetchdow'""","""'dtd'""","""'ALLminpA'""","""'ALLminpA'l1""","""'ALLminpA'l2""","""'ALLminpA'l7""","""'ALLavgpA'""",...,"""fdisoweekday4""","""fdisoweekday5""","""fdisoweekday6""","""fdisoweekday7""",LBL+ALLminpA+bt7d_000,LBL+ALLminp0+bt7d_000,LBL+aDLminpA+bt7d_000,LBL+aCOminpA+bt7d_000,LBL+aFLminpA+bt7d_000,LBL+aUAminpA+bt7d_000
0,72.0,1.0,60.0,3.0,12.0,359.0,372.0,351.0,317.0,615.96,...,0.0,0.0,0.0,0.0,326.0,675.0,626.0,545.0,326.0,545.0
1,79.0,1.0,69.0,5.0,10.0,336.0,298.0,298.0,320.0,616.73,...,0.0,1.0,0.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
2,79.0,1.0,70.0,6.0,9.0,336.0,336.0,298.0,320.0,628.87,...,0.0,0.0,1.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
3,79.0,1.0,71.0,7.0,8.0,336.0,336.0,336.0,330.0,632.73,...,0.0,0.0,0.0,1.0,315.0,675.0,631.0,587.0,315.0,587.0
4,79.0,1.0,72.0,1.0,7.0,336.0,336.0,336.0,330.0,635.47,...,0.0,0.0,0.0,0.0,315.0,1182.0,720.0,720.0,315.0,720.0


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

print(X.shape,y.shape)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(296, 411) (296, 6)


In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 296
    p = 6

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

## Bidirectional Model


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
#from keras.backend.tensorflow_backend import clear_session
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 4
#clear_session()

In [8]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='glorot_uniform'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

## Forward Ensemble

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_forward = []

    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(y_t.shape[1]):
        print("Training chain node ",i)
        y_now = y_t[:,[i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_forward.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_fwd.append(chain_forward)

TRAINING ENSEMBLE 0 :



Training chain node  0
Shapes:
 X = (207, 411) 
 Y = (207, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
207/207 [==============================] - 1s - loss: 1.3105 - mean_squared_error: 1.3105     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.2833 - mean_squared_error: 0.2833     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.2859 - mean_squared_error: 0.2859     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.2805 - mean_squared_error: 0.2805     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.1623 - mean_squared_error: 0.1623     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.0982 - mean_squared_error: 0.0982     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.0890 - mean_squared_error: 0.0890     
Epoch 8/500
207/207 [===================

207/207 [==============================] - 0s - loss: 0.0179 - mean_squared_error: 0.0179     
Epoch 62/500
207/207 [==============================] - 0s - loss: 0.0181 - mean_squared_error: 0.0181     
Epoch 63/500
207/207 [==============================] - 0s - loss: 0.0106 - mean_squared_error: 0.0106     
Epoch 64/500
207/207 [==============================] - 0s - loss: 0.0084 - mean_squared_error: 0.0084     
Epoch 65/500
207/207 [==============================] - 0s - loss: 0.0135 - mean_squared_error: 0.0135     
Epoch 66/500
207/207 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093     
Epoch 67/500
207/207 [==============================] - 0s - loss: 0.0084 - mean_squared_error: 0.0084     
Epoch 68/500
207/207 [==============================] - 0s - loss: 0.0080 - mean_squared_error: 0.0080     
Epoch 69/500
207/207 [==============================] - 0s - loss: 0.0086 - mean_squared_error: 0.0086     
Epoch 70/500
207/207 [===================

207/207 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 137/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014             
Epoch 138/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Epoch 139/500
207/207 [==============================] - 0s - loss: 8.9174e-04 - mean_squared_error: 8.9174e-04     
Epoch 140/500
207/207 [==============================] - 0s - loss: 0.0037 - mean_squared_error: 0.0037     
Epoch 141/500
207/207 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028             
Epoch 142/500
207/207 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 143/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 144/500
207/207 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
E

207/207 [==============================] - 0s - loss: 0.0244 - mean_squared_error: 0.0244     
Epoch 208/500
207/207 [==============================] - 0s - loss: 0.0206 - mean_squared_error: 0.0206     
Epoch 209/500
207/207 [==============================] - 0s - loss: 0.0169 - mean_squared_error: 0.0169     
Epoch 210/500
207/207 [==============================] - 0s - loss: 0.0116 - mean_squared_error: 0.0116     
Epoch 211/500
207/207 [==============================] - 0s - loss: 0.0095 - mean_squared_error: 0.0095     
Epoch 212/500
207/207 [==============================] - 0s - loss: 0.0058 - mean_squared_error: 0.0058     
Epoch 213/500
207/207 [==============================] - 0s - loss: 0.0068 - mean_squared_error: 0.0068     
Epoch 214/500
207/207 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036     
Epoch 215/500
207/207 [==============================] - 0s - loss: 0.0028 - mean_squared_error: 0.0028     
Epoch 216/500
207/207 [==========

207/207 [==============================] - 0s - loss: 5.3614e-06 - mean_squared_error: 5.3614e-06     
Epoch 278/500
207/207 [==============================] - 0s - loss: 1.0381e-05 - mean_squared_error: 1.0381e-05     
Epoch 279/500
207/207 [==============================] - 0s - loss: 1.3722e-05 - mean_squared_error: 1.3722e-05     
Epoch 280/500
207/207 [==============================] - 0s - loss: 2.0889e-05 - mean_squared_error: 2.0889e-05     
Epoch 281/500
207/207 [==============================] - 0s - loss: 3.0089e-05 - mean_squared_error: 3.0089e-05     
Epoch 282/500
207/207 [==============================] - 0s - loss: 3.5378e-05 - mean_squared_error: 3.5378e-05     
Epoch 283/500
207/207 [==============================] - 0s - loss: 2.7337e-05 - mean_squared_error: 2.7337e-05     
Epoch 284/500
207/207 [==============================] - 0s - loss: 3.5095e-05 - mean_squared_error: 3.5095e-05     
Epoch 285/500
207/207 [==============================] - 0s - loss: 3.9081e-05

...

207/207 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 215/500
207/207 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 216/500
207/207 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 217/500
207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012             
Epoch 218/500
207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 219/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 220/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015             
Epoch 221/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014             
Epoch 222/500
207/207 [==============================] - 0s - loss: 0.0011 - mean_squared_error: 0.0011             
E

207/207 [==============================] - 0s - loss: 1.7361e-04 - mean_squared_error: 1.7361e-04     
Epoch 286/500
207/207 [==============================] - 0s - loss: 1.5580e-04 - mean_squared_error: 1.5580e-04     
Epoch 287/500
207/207 [==============================] - 0s - loss: 3.0695e-04 - mean_squared_error: 3.0695e-04     
Epoch 288/500
207/207 [==============================] - 0s - loss: 2.7875e-04 - mean_squared_error: 2.7875e-04     
Epoch 289/500
207/207 [==============================] - 0s - loss: 2.2075e-04 - mean_squared_error: 2.2075e-04     
Epoch 290/500
207/207 [==============================] - 0s - loss: 3.1833e-04 - mean_squared_error: 3.1833e-04     
Epoch 291/500
207/207 [==============================] - 0s - loss: 2.4366e-04 - mean_squared_error: 2.4366e-04     
Epoch 292/500
207/207 [==============================] - 0s - loss: 4.9891e-04 - mean_squared_error: 4.9891e-04     
Epoch 293/500
207/207 [==============================] - 0s - loss: 5.7926e-04

207/207 [==============================] - 0s - loss: 0.0121 - mean_squared_error: 0.0121     
Epoch 358/500
207/207 [==============================] - 0s - loss: 0.0097 - mean_squared_error: 0.0097     
Epoch 359/500
207/207 [==============================] - 0s - loss: 0.0097 - mean_squared_error: 0.0097     
Epoch 360/500
207/207 [==============================] - 0s - loss: 0.0129 - mean_squared_error: 0.0129     
Epoch 361/500
207/207 [==============================] - 0s - loss: 0.0146 - mean_squared_error: 0.0146     
Epoch 362/500
207/207 [==============================] - 0s - loss: 0.0178 - mean_squared_error: 0.0178     
Epoch 363/500
207/207 [==============================] - 0s - loss: 0.0239 - mean_squared_error: 0.0239     
Epoch 364/500
207/207 [==============================] - 0s - loss: 0.0246 - mean_squared_error: 0.0246     
Epoch 365/500
207/207 [==============================] - 0s - loss: 0.0341 - mean_squared_error: 0.0341     
Epoch 366/500
207/207 [==========

207/207 [==============================] - 0s - loss: 1.3768e-04 - mean_squared_error: 1.3768e-04     
Epoch 431/500
207/207 [==============================] - 0s - loss: 1.1609e-04 - mean_squared_error: 1.1609e-04     
Epoch 432/500
207/207 [==============================] - 0s - loss: 1.0349e-04 - mean_squared_error: 1.0349e-04     
Epoch 433/500
207/207 [==============================] - 0s - loss: 7.6046e-05 - mean_squared_error: 7.6046e-05     
Epoch 434/500
207/207 [==============================] - 0s - loss: 8.3943e-05 - mean_squared_error: 8.3943e-05     
Epoch 435/500
207/207 [==============================] - 0s - loss: 7.9707e-05 - mean_squared_error: 7.9707e-05     
Epoch 436/500
207/207 [==============================] - 0s - loss: 4.9918e-05 - mean_squared_error: 4.9918e-05     
Epoch 437/500
207/207 [==============================] - 0s - loss: 3.5100e-05 - mean_squared_error: 3.5100e-05     
Epoch 438/500
207/207 [==============================] - 0s - loss: 4.2841e-05

207/207 [==============================] - 0s - loss: 6.9789e-07 - mean_squared_error: 6.9789e-07     
Epoch 500/500
207/207 [==============================] - 0s - loss: 1.2712e-06 - mean_squared_error: 1.2712e-06     
Training of node 5 complete




## Backward  Ensemble

In [10]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_reverse = []
    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print("Training chain node ",i)
        y_now = y_t[:,[-i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_reverse.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_rev.append(chain_reverse)

TRAINING ENSEMBLE 0 :



Training chain node  1
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
207/207 [==============================] - 0s - loss: 1.4561 - mean_squared_error: 1.4561     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.4845 - mean_squared_error: 0.4845     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.3680 - mean_squared_error: 0.3680     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.1991 - mean_squared_error: 0.1991     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.1504 - mean_squared_error: 0.1504     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.1160 - mean_squared_error: 0.1160     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.0885 - mean_squared_error: 0.0885     
Epoch 8/500
207/207 [==============================] - 0s - loss: 0.0727 - mean_squared_error: 0.0727     
Epoch 9/500
207/207 [==============================] - 0s

207/207 [==============================] - 0s - loss: 5.4452e-04 - mean_squared_error: 5.4452e-04     
Epoch 74/500
207/207 [==============================] - 0s - loss: 6.1635e-04 - mean_squared_error: 6.1635e-04     
Epoch 75/500
207/207 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021             
Epoch 76/500
207/207 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 77/500
207/207 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024             
Epoch 78/500
207/207 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 79/500
207/207 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024             
Epoch 80/500
207/207 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 81/500
207/207 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
E

207/207 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 147/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034             
Epoch 148/500
207/207 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 149/500
207/207 [==============================] - 0s - loss: 0.0017 - mean_squared_error: 0.0017     
Epoch 150/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 151/500
207/207 [==============================] - 0s - loss: 7.1630e-04 - mean_squared_error: 7.1630e-04
Epoch 152/500
207/207 [==============================] - 0s - loss: 7.8683e-04 - mean_squared_error: 7.8683e-04     
Epoch 153/500
207/207 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020             
Epoch 154/500
207/207 [==============================] - 0s - loss: 0.0039 - mean_squared_error: 0.0039     
Epoch 

207/207 [==============================] - 0s - loss: 3.6615e-04 - mean_squared_error: 3.6615e-04     
Epoch 219/500
207/207 [==============================] - 0s - loss: 4.0111e-04 - mean_squared_error: 4.0111e-04     
Epoch 220/500
207/207 [==============================] - 0s - loss: 4.5252e-04 - mean_squared_error: 4.5252e-04     
Epoch 221/500
207/207 [==============================] - 0s - loss: 8.5102e-04 - mean_squared_error: 8.5102e-04     
Epoch 222/500
207/207 [==============================] - 0s - loss: 4.8186e-04 - mean_squared_error: 4.8186e-04     
Epoch 223/500
207/207 [==============================] - 0s - loss: 8.6376e-04 - mean_squared_error: 8.6376e-04     
Epoch 224/500
207/207 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016             
Epoch 225/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015     
Epoch 226/500
207/207 [==============================] - 0s - loss: 0.0023 - mean_squa

207/207 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023             
Epoch 291/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 292/500
207/207 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 293/500
207/207 [==============================] - 0s - loss: 0.0073 - mean_squared_error: 0.0073     
Epoch 294/500
207/207 [==============================] - 0s - loss: 0.0059 - mean_squared_error: 0.0059     
Epoch 295/500
207/207 [==============================] - 0s - loss: 0.0043 - mean_squared_error: 0.0043     
Epoch 296/500
207/207 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 297/500
207/207 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 298/500
207/207 [==============================] - 0s - loss: 0.0049 - mean_squared_error: 0.0049     
Epoch 299/500
207/207 [==

...

207/207 [==============================] - 0s - loss: 1.7848e-05 - mean_squared_error: 1.7848e-05     
Epoch 198/500
207/207 [==============================] - 0s - loss: 2.0473e-05 - mean_squared_error: 2.0473e-05     
Epoch 199/500
207/207 [==============================] - 0s - loss: 2.4033e-05 - mean_squared_error: 2.4033e-05     
Epoch 200/500
207/207 [==============================] - 0s - loss: 2.7457e-05 - mean_squared_error: 2.7457e-05     
Epoch 201/500
207/207 [==============================] - 0s - loss: 2.6682e-05 - mean_squared_error: 2.6682e-05     
Epoch 202/500
207/207 [==============================] - 0s - loss: 1.4523e-05 - mean_squared_error: 1.4523e-05     
Epoch 203/500
207/207 [==============================] - 0s - loss: 2.1061e-05 - mean_squared_error: 2.1061e-05     
Epoch 204/500
207/207 [==============================] - 0s - loss: 1.1387e-05 - mean_squared_error: 1.1387e-05     
Epoch 205/500
207/207 [==============================] - 0s - loss: 9.9655e-06

207/207 [==============================] - 0s - loss: 5.3453e-08 - mean_squared_error: 5.3453e-08     
Epoch 267/500
207/207 [==============================] - 0s - loss: 8.2534e-08 - mean_squared_error: 8.2534e-08     
Epoch 268/500
207/207 [==============================] - 0s - loss: 9.3514e-08 - mean_squared_error: 9.3514e-08     
Epoch 269/500
207/207 [==============================] - 0s - loss: 1.6359e-07 - mean_squared_error: 1.6359e-07     
Epoch 270/500
207/207 [==============================] - 0s - loss: 1.7223e-07 - mean_squared_error: 1.7223e-07     
Epoch 271/500
207/207 [==============================] - 0s - loss: 1.1568e-07 - mean_squared_error: 1.1568e-07     
Epoch 272/500
207/207 [==============================] - 0s - loss: 1.9761e-07 - mean_squared_error: 1.9761e-07     
Epoch 273/500
207/207 [==============================] - 0s - loss: 1.3718e-07 - mean_squared_error: 1.3718e-07     
Epoch 274/500
207/207 [==============================] - 0s - loss: 4.0439e-07

207/207 [==============================] - 0s - loss: 1.2171e-06 - mean_squared_error: 1.2171e-06     
Epoch 337/500
207/207 [==============================] - 0s - loss: 1.4300e-06 - mean_squared_error: 1.4300e-06     
Epoch 338/500
207/207 [==============================] - 0s - loss: 1.8303e-06 - mean_squared_error: 1.8303e-06     
Epoch 339/500
207/207 [==============================] - 0s - loss: 3.3381e-06 - mean_squared_error: 3.3381e-06     
Epoch 340/500
207/207 [==============================] - 0s - loss: 4.8959e-06 - mean_squared_error: 4.8959e-06     
Epoch 341/500
207/207 [==============================] - 0s - loss: 5.4783e-06 - mean_squared_error: 5.4783e-06     
Epoch 342/500
207/207 [==============================] - 0s - loss: 5.6175e-06 - mean_squared_error: 5.6175e-06     
Epoch 343/500
207/207 [==============================] - 0s - loss: 3.7997e-06 - mean_squared_error: 3.7997e-06     
Epoch 344/500
207/207 [==============================] - 0s - loss: 3.0083e-06

207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 407/500
207/207 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024             
Epoch 408/500
207/207 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 409/500
207/207 [==============================] - 0s - loss: 0.0026 - mean_squared_error: 0.0026     
Epoch 410/500
207/207 [==============================] - ETA: 0s - loss: 0.0015 - mean_squared_error: 0.00 - 0s - loss: 0.0285 - mean_squared_error: 0.0285     
Epoch 411/500
207/207 [==============================] - 0s - loss: 0.0319 - mean_squared_error: 0.0319     
Epoch 412/500
207/207 [==============================] - 0s - loss: 0.0373 - mean_squared_error: 0.0373     
Epoch 413/500
207/207 [==============================] - 0s - loss: 0.0426 - mean_squared_error: 0.0426     
Epoch 414/500
207/207 [==============================] - 0s - loss: 0.0531 - mean_

207/207 [==============================] - 0s - loss: 1.6857e-04 - mean_squared_error: 1.6857e-04     
Epoch 479/500
207/207 [==============================] - 0s - loss: 2.0499e-04 - mean_squared_error: 2.0499e-04     
Epoch 480/500
207/207 [==============================] - 0s - loss: 1.9027e-04 - mean_squared_error: 1.9027e-04     
Epoch 481/500
207/207 [==============================] - 0s - loss: 2.9153e-04 - mean_squared_error: 2.9153e-04     
Epoch 482/500
207/207 [==============================] - 0s - loss: 0.0014 - mean_squared_error: 0.0014             
Epoch 483/500
207/207 [==============================] - 0s - loss: 0.0085 - mean_squared_error: 0.0085     
Epoch 484/500
207/207 [==============================] - 0s - loss: 0.0091 - mean_squared_error: 0.0091     
Epoch 485/500
207/207 [==============================] - 0s - loss: 0.0058 - mean_squared_error: 0.0058     
Epoch 486/500
207/207 [==============================] - 0s - loss: 0.0039 - mean_squared_error: 0.003

## Mean Aggregation

In [11]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


ENSEMBLE 2:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


In [12]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [13]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [14]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  41.99168903348254
MAE:  24.86156737938356
R_Squared:  0.7414026620455786
Adjusted R-squared -  0.736033859181473
